In [1]:
import ROOT

In [ ]:
def compute_response_matrix(gen_mttbar_edges, gen_theta_edges, gen_coefficient_edges, reco_devisions, gen_step8_mttbar, gen_step0_mttbar, step8_mttbar, gen_step8_scat_angle, gen_step0_scat_angle, step8_scat_angle, gen_step8_coefficient, gen_step0_coefficient, step8_coefficient, gen_step8_weights, gen_step0_weights, step8_weights, var_name):
    reco_mttbar_edges = []
    reco_theta_edges = []
    reco_coefficient_edges = []

    for i in range(len(gen_mttbar_edges) - 1):
        delta_mttbar = gen_mttbar_edges[i + 1] - gen_mttbar_edges[i]
        reco_mttbar_edges += [gen_mttbar_edges[i] + (j * delta_mttbar / reco_devisions) for j in range(reco_devisions)]
    reco_mttbar_edges += [gen_mttbar_edges[-1]]
    
    for i in range(len(gen_theta_edges) - 1):
        delta_theta = gen_theta_edges[i + 1] - gen_theta_edges[i]
        reco_theta_edges += [gen_theta_edges[i] + (j * delta_theta / reco_devisions) for j in range(reco_devisions)]
    reco_theta_edges += [gen_theta_edges[-1]]
    
    for i in range(len(gen_coefficient_edges) - 1):
        delta_coefficient = gen_coefficient_edges[i + 1] - gen_coefficient_edges[i]
        reco_coefficient_edges += [gen_coefficient_edges[i] + (j * delta_coefficient / reco_devisions) for j in range(reco_devisions)]
    reco_coefficient_edges += [gen_coefficient_edges[-1]]
    
    num_gen_mttbar_bins = len(gen_mttbar_edges) - 1
    num_gen_scat_angle_bins = len(gen_theta_edges) - 1
    num_gen_coefficient_bins = len(gen_coefficient_edges) - 1

    num_reco_mttbar_bins = len(reco_mttbar_edges) - 1
    num_reco_scat_angle_bins = len(reco_theta_edges) - 1
    num_reco_coefficient_bins = len(reco_coefficient_edges) - 1
    
    response_matrix = (
        hist.new
        .Variable(gen_mttbar_edges, name='gen mttbar')
        .Variable(reco_mttbar_edges, name='reco mttbar')
        .Variable(gen_theta_edges, name='gen scat angle')
        .Variable(reco_theta_edges, name='reco scat angle')
        .Variable(gen_coefficient_edges, name='gen ' + var_name)
        .Variable(reco_coefficient_edges, name='reco ' + var_name)
        .Weight()
    )
    # fill the reconstructed events
    response_matrix.fill(gen_step8_mttbar, step8_mttbar, gen_step8_scat_angle, step8_scat_angle, gen_step8_coefficient, step8_coefficient, weight=step8_weights)
    
    # fill the generator level events in the reconstruction underflow bin
    # this is back at step0 so that we can unfold back to full phase space
    response_matrix.fill(gen_step0_mttbar, np.zeros(len(gen_step0_mttbar)), gen_step0_scat_angle, np.zeros(len(gen_step0_scat_angle)), gen_step0_coefficient, np.zeros(len(gen_step0_coefficient)), weight=gen_step0_weights)
    
    
    # fill the generator level events at step8 that we weren't able to reconstruct
    # these are the events that have eventWeight < trueLevelWeight at step8
    
    # first fill the gen weights
    response_matrix.fill(gen_step8_mttbar, np.zeros(len(gen_step8_mttbar)), gen_step8_scat_angle, np.zeros(len(gen_step8_scat_angle)), gen_step8_coefficient, np.zeros(len(gen_step8_coefficient)), weight=gen_step8_weights)
    
    # next create a separate response matrix to fill the reco weights
    # this is because we need to subtract the entries and variances and can't
    # do that with a "simple" negative weight instead of it being positive
    # (variances are sum of weights squared)
    _response_matrix = (
        hist.new
        .Variable(gen_mttbar_edges, name='gen mttbar')
        .Variable(reco_mttbar_edges, name='reco mttbar')
        .Variable(gen_theta_edges, name='gen scat angle')
        .Variable(reco_theta_edges, name='reco scat angle')
        .Variable(gen_coefficient_edges, name='gen ' + var_name)
        .Variable(reco_coefficient_edges, name='reco ' + var_name)
        .Weight()
    )
    
    # fill this new matrix
    _response_matrix.fill(step8_mttbar, np.zeros(len(step8_mttbar)), step8_scat_angle, np.zeros(len(step8_scat_angle)), step8_coefficient, np.zeros(len(step8_coefficient)), weight=step8_weights)
    
    # set values and variances equal to the difference between response_matrix
    # and _response_matrix to account for these events that aren't reconstructed
    response_matrix[:, :, :, :, :, :] = np.concatenate(((response_matrix.values(flow=True) - _response_matrix.values(flow=True))[:, :, :, :, :, :, None], (response_matrix.variances(flow=True) - _response_matrix.variances(flow=True))[:, :, :, :, :, :, None]), axis=6)
    
    entries = response_matrix.values(flow=True)
    errors = np.sqrt(response_matrix.variances(flow=True))
    
    h_response_matrix = ROOT.TH2F('', 'Response Matrix for '+ var_name + '; reco #frac{2#Theta}{#pi}, m_{ttbar}, ' + var_name + '; gen #frac{2#Theta}{#pi}, m_{ttbar}, ' + var_name + ' ', num_reco_scat_angle_bins * num_reco_mttbar_bins * num_reco_coefficient_bins, 0, 1, num_gen_scat_angle_bins * num_gen_mttbar_bins * num_gen_coefficient_bins, 0, 1)
    h_response_matrix.Sumw2()

    for ith_gen_mttbar_bin in range(len(gen_mttbar_edges) + 1):
        for jth_reco_mttbar_bin in range(len(reco_mttbar_edges) + 1):
            for ith_gen_scat_angle_bin in range(num_gen_scat_angle_bins):
                for jth_reco_scat_angle_bin in range(num_reco_scat_angle_bins):
                    for ith_gen_coef_bin in range(num_gen_coefficient_bins):
                        for jth_reco_coef_bin in range(num_reco_coefficient_bins):
                            entry = entries[ith_gen_mttbar_bin][jth_reco_mttbar_bin][ith_gen_scat_angle_bin + 1][jth_reco_scat_angle_bin + 1][ith_gen_coef_bin + 1][jth_reco_coef_bin + 1]
                            error = errors[ith_gen_mttbar_bin][jth_reco_mttbar_bin][ith_gen_scat_angle_bin + 1][jth_reco_scat_angle_bin + 1][ith_gen_coef_bin + 1][jth_reco_coef_bin + 1]
                            if ith_gen_mttbar_bin == 0:
                                bin_x = 0
                            else:
                                bin_x = (ith_gen_mttbar_bin - 1) * num_gen_scat_angle_bins * num_gen_coefficient_bins + ith_gen_scat_angle_bin * num_gen_coefficient_bins + ith_gen_coef_bin + 1

                            if jth_reco_mttbar_bin == 0:
                                bin_y = 0
                            else:
                                bin_y = (jth_reco_mttbar_bin - 1) * num_reco_scat_angle_bins * num_reco_coefficient_bins + jth_reco_scat_angle_bin * num_reco_coefficient_bins + jth_reco_coef_bin + 1

                            h_response_matrix.SetBinContent(bin_y, bin_x, entry)
                            h_response_matrix.SetBinError(bin_y, bin_x, error)

    c_response_matrix = ROOT.TCanvas('','',2100,2100)
    c_response_matrix.SetLogz()
    c_response_matrix.SetLeftMargin(0.15)
    c_response_matrix.cd(1)
    c_response_matrix.SetRightMargin(0.15)
    h_response_matrix.SetStats(0)
    h_response_matrix.Draw('colz')
    c_response_matrix.Draw()
    h_response_matrix.SaveAs(var_name + '_response_matrix.root')
    c_response_matrix.SaveAs(var_name + '_response_matrix.pdf')
    return h_response_matrix

In [ ]:
unfold = ROOT.TUnfold(h_response_matrix, ROOT.TUnfold.kHistMapOutputVert, ROOT.TUnfold.kRegModeNone)

if unfold.SetInput(h_response_matrix.ProjectionX()) >= 10000:
    print(f'Error!!')

tauMin=0.0
tauMax=0.0
nScan=3
logTauX = ROOT.TSpline3()
logTauY = ROOT.TSpline3()
lCurve  = ROOT.TGraph()

unfold.DoUnfold(0.0)

optimal_tau = unfold.GetTau()
chi_squared = (unfold.GetChi2A() + unfold.GetChi2L())/ unfold.GetNdf()

nGen = num_gen_scat_angle_bins * num_gen_mttbar_bins * num_gen_coefficient_bins
nDet = num_reco_scat_angle_bins * num_reco_mttbar_bins * num_reco_coefficient_bins

binMap = np.zeros(nGen+2, dtype=np.int32)
for i in range(1, nGen+1):
    binMap[i] = i
binMap[0] = -1
binMap[nGen+1] = -1

histMunfold= ROOT.TH1D("Unfolded","Generator-level Dists",nGen,0,1)
unfold.GetOutput(histMunfold,binMap)
histMdetFold= ROOT.TH1D("FoldedBack","Reco-level Dists",nDet,0,1)
unfold.GetFoldedOutput(histMdetFold)

# store global correlation coefficients
histRhoi= ROOT.TH1D("rho_I","Global Correlation Coefficients",nGen,0,1)

unfold.GetRhoI(histRhoi,binMap)